# Document Parsing

In this notebook, we will be iterating through all the links created in file 2.2. (i.e. link_dict.pickle), extracting the necessary information, and then saving an updated dictionary, which will be analyzed in section 3. 

In [64]:
import numpy as np
import pandas as pd
import os
import pickle
import time
import json
import requests
import re

from bs4 import BeautifulSoup

In [4]:
with open('../Step1-Data/2-link_dict.pickle', 'rb') as f:
    link_dict = pickle.load(f)

In [11]:
# Sanity check
list(link_dict.keys())[:5]

['0000006201', '0001158449', '0000320193', '0001551152', '0001140859']

# Single Name Sample

First, we will start by using a single name to parse its documents, then extrapolate to all.

In [12]:
sample = link_dict['0000006201']
sample

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,Link
49,0000006201-21-000014,2021-02-17,2020-12-31,2021-02-17T17:17:57.000Z,34,10-K,001-08400,21646186,,43925703,1,1,aal-20201231.htm,10-K 2020 02.17.21,https://www.sec.gov/Archives/edgar/data/6201/0...
150,0000006201-20-000023,2020-02-19,2019-12-31,2020-02-19T07:31:30.000Z,34,10-K,001-08400,20627428,,30851334,1,1,a10k123119.htm,10-K 2019 02.19.20,https://www.sec.gov/Archives/edgar/data/6201/0...
225,0000006201-19-000009,2019-02-25,2018-12-31,2019-02-25T07:31:34.000Z,34,10-K,001-08400,19628071,,30572408,1,0,a10k123118.htm,10-K 2018 02.25.19,https://www.sec.gov/Archives/edgar/data/6201/0...
315,0000006201-18-000009,2018-02-21,2017-12-31,2018-02-21T08:02:40.000Z,34,10-K,001-08400,18627088,,27914491,1,0,a10k123117.htm,10-K,https://www.sec.gov/Archives/edgar/data/6201/0...
412,0001193125-17-051216,2017-02-22,2016-12-31,2017-02-22T08:01:43.000Z,34,10-K,001-08400,17627073,,24888480,1,0,d286458d10k.htm,FORM 10-K,https://www.sec.gov/Archives/edgar/data/6201/0...
538,0001193125-16-474605,2016-02-24,2015-12-31,2016-02-24T08:04:10.000Z,34,10-K,001-08400,161450518,,26170400,1,0,d78287d10k.htm,FORM 10-K,https://www.sec.gov/Archives/edgar/data/6201/0...
651,0001193125-15-061145,2015-02-25,2014-12-31,2015-02-25T08:02:34.000Z,34,10-K,001-08400,15645918,,39524925,1,0,d829913d10k.htm,FORM 10-K,https://www.sec.gov/Archives/edgar/data/6201/0...
750,0000006201-14-000004,2014-02-28,2013-12-31,2014-02-28T07:52:16.000Z,34,10-K,001-08400,14651496,,47888955,1,0,aagaa10k-20131231.htm,10-K,https://www.sec.gov/Archives/edgar/data/6201/0...


In [15]:
sample_link = sample['Link'].values[0]
sample_link

'https://www.sec.gov/Archives/edgar/data/6201/000000620121000014/aal-20201231.htm'

In [26]:
# Used for the requests
heads = {#'Host': 'www.sec.gov', 
         #'Connection': 'close',
         'Accept': 'application/json',#, text/javascript, */*; q=0.01', 
         'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
         }

In [54]:
response = requests.get(sample_link, headers=heads).text

In [55]:
response[:500]

'<?xml version="1.0" ?><!--XBRL Document Created with Wdesk from Workiva--><!--Copyright 2021 Workiva--><!--r:e284921f-ba97-4934-aa95-4c28f5345f62,g:0d6c1504-a6c8-4292-b9c4-47751639beae,d:218984ca4fa54d3589b92be28c18f351--><html xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:srt="http://fasb.org/srt/2020-01-31" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/1999/xhtml" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xml'

In [58]:
soup = BeautifulSoup(response, 'lxml')

In [148]:
cleaned_soup = soup.text

In [149]:
raw_10k = str(soup)

In [101]:
locs7 = re.finditer(r"ITEM\s7(|\.) Management", cleaned_soup, re.IGNORECASE)
[x for x in locs7]

[<re.Match object; span=(158292, 158310), match='Item 7. Management'>,
 <re.Match object; span=(167163, 167181), match='Item\xa07. Management'>,
 <re.Match object; span=(180987, 181005), match='Item\xa07. Management'>,
 <re.Match object; span=(430418, 430436), match='Item 7. Management'>,
 <re.Match object; span=(440363, 440381), match='Item 7. Management'>,
 <re.Match object; span=(451837, 451855), match='Item 7. Management'>]

In [103]:
locs8 = re.finditer(r"ITEM\s8(|A)(|\.)\s(Consolidated|Financial)", cleaned_soup, re.IGNORECASE)
[x for x in locs8]

[<re.Match object; span=(502641, 502662), match='ITEM\xa08A.\xa0CONSOLIDATED'>]

I wonder if maybe it would be best to use some sort of clustering algorithm to find the correct locations? 

In [215]:
start = time.time()
link_dict2 = {}

for key in list(link_dict.keys()):

    loc7list = []
    loc8list = []
    
    sample = link_dict[key].copy()
    for i in range(len(sample)):
        sample_link = sample.iloc[i,:]['Link']
        response = requests.get(sample_link, headers=heads).text
        soup = BeautifulSoup(response, 'lxml')
        cleaned_soup = soup.text
        locs7 = re.finditer(r"ITEM\s7(|\.)\sManagement", cleaned_soup, re.IGNORECASE)
        locs8 = re.finditer(r"ITEM\s8(|A)(|\.)\s(Consolidated|Financial)", cleaned_soup, re.IGNORECASE)

        loc7list.append(locs7)
        loc8list.append(locs8)
    
    sample['Loc7'] = loc7list
    sample['Loc8'] = loc8list
    
    link_dict2[key] = sample.copy()

end = time.time()

KeyboardInterrupt: 

In [ ]:
with open('2-link_dict.pickle', 'wb') as handle:
    pickle.dump(link_dict2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [218]:
print(time.time()-start)

30001.40250992775


In [220]:
len(link_dict2.keys())

686

In [214]:
((end-start) * 1110)/5 /60/60

6.319669485092163

In [205]:
[x for x in link_dict2['0001158449']['Loc7'].values[0]]

[<re.Match object; span=(92292, 92310), match='Item 7. Management'>,
 <re.Match object; span=(105990, 106008), match='Item 7. Management'>]

In [193]:
[x for x in all_locs[4][0]]

[]

In [196]:
link_dict.keys()[:5]

TypeError: 'dict_keys' object is not subscriptable